<a href="https://colab.research.google.com/github/praveen2003-pk/task1/blob/main/secure_user_authentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-login flask-bcrypt pyngrok


In [ ]:
from flask import Flask, request, redirect, url_for, render_template_string
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
from pyngrok import ngrok


app = Flask(__name__)
app.secret_key = "super_secret_key"

bcrypt = Bcrypt(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = "login"


ngrok.set_auth_token("38IuU34N9a94gmLP0IvIGl3pPvU_3KDTEiDnczTp7SPScLFXo")


users = {}


class User(UserMixin):
    def __init__(self, username, password):
        self.id = username
        self.password = password

@login_manager.user_loader
def load_user(user_id):
    return users.get(user_id)


@app.route('/')
def home():
    return '''
    <h2>Secure User Authentication System</h2>
    <a href="/register">Register</a><br>
    <a href="/login">Login</a>
    '''


@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = bcrypt.generate_password_hash(request.form['password']).decode('utf-8')
        users[username] = User(username, password)
        return redirect(url_for('login'))

    return render_template_string("""
    <h2>Register</h2>
    <form method="POST">
        <input name="username" placeholder="Username" required><br><br>
        <input name="password" type="password" placeholder="Password" required><br><br>
        <button type="submit">Register</button>
    </form>
    """)


@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = users.get(username)

        if user and bcrypt.check_password_hash(user.password, password):
            login_user(user)
            return redirect(url_for('dashboard'))
        return "Invalid username or password"

    return render_template_string("""
    <h2>Login</h2>
    <form method="POST">
        <input name="username" placeholder="Username" required><br><br>
        <input name="password" type="password" placeholder="Password" required><br><br>
        <button type="submit">Login</button>
    </form>
    """)


@app.route('/dashboard')
@login_required
def dashboard():
    return f"""
    <h2>Welcome {current_user.id}</h2>
    <p>This is a protected page.</p>
    <a href="/logout">Logout</a>
    """


@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))


public_url = ngrok.connect(5000)
print("Public URL:", public_url)

app.run(port=5000)


INFO:werkzeug:127.0.0.1 - - [15/Jan/2026 18:41:03] "GET / HTTP/1.1" 200 -
